In [2]:
import pandas as pd
import ast
from psycopg2.extras import execute_values
from db_connection import get_connection
from datetime import datetime

In [3]:
def parse_date(date_str):
    if pd.isna(date_str):
        return None
    try:
        # 嘗試完整日期
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        try:
            # 嘗試年-月格式
            return datetime.strptime(date_str, "%Y-%m").date()
        except ValueError:
            try:
                # 嘗試年份格式
                return datetime.strptime(date_str, "%Y").date()
            except ValueError:
                # 無法解析的格式
                return None

In [4]:
books_df = pd.read_csv('dataset/books_data.csv')
books_df.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [5]:
books_df.columns = [
    'title', 'description', 'authors', 'cover_url', 'preview_link', 
    'publisher', 'published_date', 'info_link', 'categories', 'ratings_count'
]
#Change ratings_count NaN to 0
books_df['ratings_count'] = books_df['ratings_count'].fillna(int(0))
books_df.head()

,title,description,authors,cover_url,preview_link,publisher,published_date,info_link,categories,ratings_count
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],0.0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],0.0
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],0.0
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],0.0
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,0.0


In [6]:
# Data cleaning
def safe_eval(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) else []
    except (ValueError, SyntaxError):
        return []

books_df['authors'] = books_df['authors'].apply(safe_eval)
books_df['categories'] = books_df['categories'].apply(safe_eval)

# 移除空白，避免錯誤的名稱出現
books_df['authors'] = books_df['authors'].apply(lambda x: [author.strip() for author in x])
books_df['categories'] = books_df['categories'].apply(lambda x: [cat.strip() for cat in x])

books_df.head()

,title,description,authors,cover_url,preview_link,publisher,published_date,info_link,categories,ratings_count
0,Its Only Art If Its Well Hung!,NaN,[Julie Strain],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,[Comics & Graphic Novels],0.0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,[Philip Nel],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,[Biography & Autobiography],0.0
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,[David R. Ray],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,[Religion],0.0
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,[Veronica Haddon],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,[Fiction],0.0
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,[Edward Long],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,[],0.0


In [7]:
books_df['published_date'] = books_df['published_date'].apply(parse_date)

In [8]:
insert_books = """
    INSERT INTO books (
        title, description, cover_url, preview_link, info_link,
        publisher, published_date, categories, ratings_count
    ) VALUES %s
    ON CONFLICT (title) DO NOTHING;
"""

book_values = [(
    row.title, row.description, row.cover_url, row.preview_link, row.info_link,
    row.publisher, row.published_date, row.categories, row.ratings_count
) for _, row in books_df.iterrows()]

with get_connection() as conn:
    with conn.cursor() as cur:
        execute_values(cur, insert_books, book_values)
        conn.commit()
    print("Books 資料已成功匯入")

Books 資料已成功匯入


In [9]:
# Start inserting authors

with get_connection() as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT name, author_id FROM authors;")
        existing_authors = {name: author_id for name, author_id in cur.fetchall()}           

In [10]:
all_authors = set()
for authors in books_df['authors']:
    all_authors.update(authors)

new_authors = [(author,) for author in all_authors if author not in existing_authors]

if new_authors:
    insert_authors = """
        INSERT INTO authors (name)
        VALUES %s
        ON CONFLICT (name) DO NOTHING;
    """

    with get_connection() as conn:
        with conn.cursor() as cur:
            execute_values(cur, insert_authors, new_authors)
            conn.commit()
        print(f"{len(new_authors)} 位新作者已成功匯入到 authors 表格")

In [11]:
with get_connection() as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT name, author_id FROM authors;")
        existing_authors = {name: author_id for name, author_id in cur.fetchall()}

In [12]:
def insert_book_authors():
    with get_connection() as conn:
        with conn.cursor() as cur:
            # 取得書名到 book_id 的 mapping
            cur.execute("SELECT book_id, title FROM books;")
            book_map = {title: book_id for book_id, title in cur.fetchall()}

            # 插入 book_authors 
            authors_data = []
            for _, row in books_df.iterrows():
                book_id = book_map.get(row['title'])
                if book_id:
                    for seq, author in enumerate(row['authors']):
                        author_id = existing_authors.get(author)
                        if author_id:
                            authors_data.append((book_id, author_id, seq))

            execute_values(cur, """
                INSERT INTO book_authors (book_id, author_id, seq)
                VALUES %s
                ON CONFLICT DO NOTHING;
            """, authors_data)
            conn.commit()
        print(f"{len(authors_data)} 筆資料已成功匯入到 book_authors 關聯表")
        
insert_book_authors()

230102 筆資料已成功匯入到 book_authors 關聯表
